In [7]:
#######
# This cell is meant to install packages if some are missing
#######

# !pip install tqdm scipy
# !pip install matplotlib
# !pip install opencv-contrib-python

ERROR: Could not find a version that satisfies the requirement threading (from versions: none)
ERROR: No matching distribution found for threading


In [1]:

import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import time
from time import sleep
import numpy as np
from tqdm import tqdm
from kalmanfilter import KalmanFilter
import cv2
import math

import aruco_vision as av

In [ ]:
### Connect the thymio(the speed set to 0 is here in case of an error during calibration or during the code)
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

SPEED = 0
    
def motors(left, right):
    return {
        "motor.left.target": [left],
        "motor.right.target": [right],
}

await node.wait_for_variables()
node.send_set_variables(motors(SPEED, SPEED))

# Calibration

In this section, we will run multiple functions, in order to calibrate our thymio to the current environement. This gives us more flexibility, and makes it easier to use different thymio in different environment.

### Calibration of the thymio 

In this first step, we will use the following image to calibrate the speed, its conversion ratio from the thymio's sensor to mm/s, and its variance for the Kalman filter.
<!-- ![localisation map](Pictures/Picture3.jpg) -->
<img src="Pictures/Picture1.jpg" width="900"/>

In this next function, we will ask the thymio to move forward, at a known speed. As soon as its ground sensor detect it is on the black line, it will start a timer. When it exit the line, the timer will stop, the thymio will stop aswell, and all the desired values will be computed. In this function, the line length is know and we need to be careful to properly align the thymio as to make it follow the line. We can also adjust the speed to make it go as straight as possible, and adjust the transition threshold if the ligth's intensity in the current room requires it.

In [8]:

thymio_data = []
SPEED = 60
Ts = 0.01
TRANSITION_THRESHOLD = 400
LINE_LENGTH = 300

timerStarted = False
onLine = True
start = 0
end = 0
iter = 0
startIter = 0
duration = 0
speed_conversion = 0
q_nu = 0

def get_data():
    thymio_data.append({"ground":list(node["prox.ground.reflected"]),
                    "left_speed":node["motor.left.speed"],
                    "right_speed":node["motor.right.speed"]})

def compute_data(start, end, startIter):
    duration = end - start
    speed_mms = LINE_LENGTH / duration
    speed_conversion = speed_mms / SPEED
    avg_speed = [(x["left_speed"]+x["right_speed"])/2/speed_conversion for x in thymio_data]
    q_nu = np.std(avg_speed[startIter:])/2
    print(f"The conversion factor for the speed of the thymio in mm/s is : {speed_conversion} ")
    print(f"With a desired speed of : {SPEED}, the thymio speed is : {speed_mms} mm/s")
    print(f"The standard deviation from the speed state (q_nu) and speed measurement (r_nu) is : {q_nu} ")


await node.wait_for_variables()
node.send_set_variables(motors(SPEED+1, SPEED))

while(onLine == True):
    await node.wait_for_variables()
    get_data()
    await client.sleep(Ts)
    #tenter avec un average des 2 capteurs
    # avg_gnd = np.mean(thymio_data[iter]["ground"])
    # print(avg_gnd)
    if(thymio_data[iter]["ground"][0] < TRANSITION_THRESHOLD):
        if(timerStarted == False):
            timerStarted = True
            start = time.time()
            startIter = iter
    elif(timerStarted == False):
        pass
    else:
        end = time.time()
        await node.wait_for_variables()
        node.send_set_variables(motors(0, 0))
        await client.sleep(Ts)
        onLine = False
        compute_data(start, end, startIter)
    iter = iter + 1



NameError: name 'node' is not defined

### Calibration of the camera
In this next cell, we will retrieve the the grid from the camera, and do all the conversion requiered for the rest of the project. This is done here, because we need to calibrate the image aswell, according to the position of the webcam. This function will make a conversion ratio between the image size and the known size of our grid.

In [2]:

vision = av.ArucoVision()
print(vision.grid[1][1])



0.0


In [ ]:
Ici on pourrait éventuellement calculer la taille de chaque case, et faire une variance pour kalman

## Run code

Now that we have everything set up, we can run the code

In [9]:
#######
# The following values have been set in the calibration part
# 
# Ts = 0.01
# Speed = 60
# TRANSITION_THRESHOLD = 400
# If a preview is needed, run the following command
#
# print(f" The sampling time is : {Ts} s \n The speed is : {SPEED} mm/s \n The transition threshold is : {TRANSITION_THRESHOLD} \n")¨
# 
#######
#  The following values are the results of the calibration part 
#
# q_nu 
# r_nu
# conversion
# 
# print(f"The conversion factor for the speed of the thymio in mm/s is : {conversion} ")
# print(f"The standard deviation from the speed state (q_nu) and speed measurement (r_nu) is : {std_speed} ")
#######

r_nu = q_nu
q_p = 1
r_p = 1
vision.update_coordinates()
KF = KalmanFilter(Ts, vision.thymio_position, speed_conversion, q_p, q_nu, r_p, q_nu)  # we initialize the filter




 The sampling time is : 0.01 s 
 The speed is : 60 mm/s 
 The transition threshold is : 400 

